In [ ]:
Sys.setlocale(category = "LC_ALL", locale = "en_US.UTF-8")

# Dependencies

# Functions

In [35]:
source("/Users/anabbi/git/ped_CapTCRseq/R/Immune_Diversity.R")

# Paths

In [ ]:
datapath <- "/Users/anabbi/OneDrive - UHN/Documents/INTERCEPT/Data/"
plotpath <- "/Users/anabbi/OneDrive - UHN/Documents/INTERCEPT/Plots/"
manifestpath <- "/Users/anabbi/OneDrive - UHN/Documents/INTERCEPT/Manifests/"

gitpath <- "/Users/anabbi/git/ped_CapTCRseq/"

# Main

In [ ]:
h4hpath <- "/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/"

## TRB

In [ ]:
file_list <- list.files(h4hpath, pattern = paste0("CLONES_", "TRB"), recursive = T)

In [ ]:
tail(file_list)

In [ ]:
# remove downsample files
file_list <- file_list[!grepl("_2000000", file_list)]

Some files are duplicated. Make a table to identify and pick the larger clone file

In [ ]:
file_df <- as.data.frame(file_list)

In [ ]:
file_df$samplename <- gsub(".*CLONES_TRB", "", file_df$file_list)
file_df$samplename <- gsub(".txt", "", file_df$samplename)

In [ ]:
file_df$size <- NA
for(i in 1:nrow(file_df)){
    file_df$size[i] <- file.size(paste0(h4hpath,file_df$file_list[i]))
}

In [ ]:
head(file_df)

In [ ]:
file_df[ grepl("383-03", file_df$file_list),]

In [ ]:
file_df <- file_df[ !grepl("immunarch", file_df$file_list),]

In [ ]:
#if duplicated keep the bigger file corresponding to deeper seqeuncing.
file_df_dedup <- file_df[0,]

for(s in unique(file_df$samplename)){
    tmp <- file_df[ file_df$samplename == s,]
    if(nrow(tmp) ==1 ){
        file_df_dedup <- rbind(file_df_dedup, tmp)
    }
    if(nrow(tmp) > 1 ){
        file_df_dedup <- rbind(file_df_dedup, tmp[ tmp$size == max(tmp$size),])
    }    
}

In [ ]:
file_df_dedup[ grepl("343-05", file_df_dedup$file_list),]

In [ ]:
dim(file_df)
dim(file_df_dedup)

In [ ]:
file_df_dedup_trb <- file_df_dedup
save(file_df_dedup_trb, file = paste0(datapath,"capTCRseq/TRBfiles.RData"))

In [ ]:
batchTRB <- immunelistfx(file_df_dedup$file_list, h4hpath, "TRB")

In [ ]:
names(batchTRB) <- gsub(".*CLONES_TRB", "", names(batchTRB))

In [ ]:
head(names(batchTRB))

In [ ]:
TRBlist <- batchTRB 

In [ ]:
save(TRBlist, file = paste0(datapath,"capTCRseq/TRBlist.RData"))

### divstats

In [33]:
save(TRBlist, file = "/Users/anabbi/Desktop/H4H/INTERCEPT/analysis/metadata/TRBlist.RData")
save(file_df_dedup_trb, file = "/Users/anabbi/Desktop/H4H/INTERCEPT/analysis/metadata/TRBfiles.RData")

### Cleanup divstats

In [ ]:
divstats <- read.csv("/Users/anabbi/Desktop/H4H/INTERCEPT/analysis/metadata/divstats_TRBCHP.csv"), header = T, stringsAsFactors = F, row.names = 1)

In [ ]:
head(rownames(divstats))

In [ ]:
head(divstats)

In [ ]:
#add 9 samples with nonproductive cdr3
addsamples <- c('CHP_347-01-cfDNA','CHP_366-05_cfDNA','CHP_377-01-cfDNA',
                'CHP_381-05_cfDNA','CHP_391-02_cfDNA','CHP_392-01_cfDNA',
                'CHP_398-01_cfDNA','CHP_401-03_cfDNA','CHP_419-FFPE-DNA_S2')

In [ ]:
divstats[ nrow(divstats) + 9 , ] <- NA

In [ ]:
rownames(divstats)[is.na(divstats$TRB)] <- addsamples

In [ ]:
#example sample_id CHP_xxx_xx
divstats$sample_id <- rownames(divstats)

#example basename CHP_xxx-xx-PBMC-DNA
divstats$basename <- rownames(divstats)

In [ ]:
# add filenames to divstats
divstats$file <- myfilenames$file_list[ match(divstats$sample_id, myfilenames$samplename)]

Separate samples

In [ ]:
#divstats$Sample <- NULL

In [ ]:
divstats$Sample <- NA
divstats$Sample[grepl("PBMC", rownames(divstats))] <- "PBMC"
divstats$Sample[grepl("cfDNA", rownames(divstats))] <- "cfDNA"
divstats$Sample[grepl("FFPE", rownames(divstats))] <- "Tumor"

Separate LFS and non-LFS

In [ ]:
divstats$Group <- NA
divstats$Group[grepl("LFS", rownames(divstats))] <- "LFS"
divstats$Group[grepl("CHP", rownames(divstats))] <- "CHP"

In [ ]:
#table(divstats$Group, useNA = "always")

In [ ]:
#tapply(divstats$TRB, divstats$sampletype, summary)

In [ ]:
summary(divstats$Reads)

Get patient ids

In [ ]:
divstats$Patient <- NA
# CHP
divstats$Patient[divstats$Group == "CHP"] <- substr(rownames(divstats)[divstats$Group == "CHP"],1,7)
# Normal samples have either YO or MO
divstats$Patient[grepl("YO", rownames(divstats))] <- gsub("_PBMC_DNA", "", rownames(divstats)[grepl("YO", rownames(divstats))])
divstats$Patient[grepl("MO", rownames(divstats))] <- gsub("_PBMC_DNA", "", rownames(divstats)[grepl("MO", rownames(divstats))])
# LFS
divstats$Patient[divstats$Group == "LFS"] <- substr(rownames(divstats)[divstats$Group == "LFS"],1,12)

Get cycles. For LFS, Normals and Tumors recode to 00

In [ ]:
divstats$Cycle <- NA
divstats$Cycle[divstats$Group == "CHP"] <- substr(rownames(divstats)[divstats$Group == "CHP"],9,10)
divstats$Cycle[grepl("YO", rownames(divstats))] <- "00"
divstats$Cycle[grepl("MO", rownames(divstats))] <- "00"
divstats$Cycle[ divstats$Sample == "Tumor"] <- "00"
divstats$Cycle[ divstats$Group == "LFS"] <- "00"

In [ ]:
divstats$sample_id <- NA

# bind patient and cycle for PBMC and cfdna to generate sample id
divstats$sample_id[ divstats$Sample != "Tumor"] <- paste(divstats$Patient[ divstats$Sample != "Tumor"], "_", 
                                                          divstats$Cycle[ divstats$Sample != "Tumor"], sep = "")

# For tumor samples start with rownames because some of them are two biopsies from the same tumor
divstats$sample_id[ divstats$Sample == "Tumor"] <- gsub("-FFPE-DN.*", "", rownames(divstats)[ divstats$Sample == "Tumor"])

In [ ]:
divstats$sample_id <- gsub("-", "_", divstats$sample_id)

In [ ]:
divstats$Patient <- NULL
divstats$Cycle <- NULL
divstats$Sample <- NULL
divstats$Group <- NULL

In [ ]:
tail(divstats,10)

In [ ]:
write.csv(divstats,
          paste0(datapath, "capTCRseq/divstats_TRBCHP.csv"), row.names = TRUE)

## TRA

In [ ]:
file_list <- list.files(h4hpath, pattern = paste0("CLONES_", "TRA"), recursive = T)

In [ ]:
tail(file_list)

In [ ]:
# remove downsample files
file_list <- file_list[!grepl("_2000000", file_list)]

Some files are duplicated. Make a table to identify and pick the larger clone file

In [ ]:
file_df <- as.data.frame(file_list)

In [ ]:
file_df$samplename <- gsub(".*CLONES_TRA", "", file_df$file_list)
file_df$samplename <- gsub(".txt", "", file_df$samplename)

In [ ]:
file_df$size <- NA
for(i in 1:nrow(file_df)){
    file_df$size[i] <- file.size(paste0(h4hpath,file_df$file_list[i]))
}

In [ ]:
head(file_df)

In [ ]:
file_df[ grepl("383-03", file_df$file_list),]

In [ ]:
file_df <- file_df[ !grepl("immunarch", file_df$file_list),]

In [ ]:
file_df_dedup <- file_df[0,]

for(s in unique(file_df$samplename)){
    tmp <- file_df[ file_df$samplename == s,]
    if(nrow(tmp) ==1 ){
        file_df_dedup <- rbind(file_df_dedup, tmp)
    }
    if(nrow(tmp) > 1 ){
        file_df_dedup <- rbind(file_df_dedup, tmp[ tmp$size == max(tmp$size),])
    }    
}

In [ ]:
file_df_dedup[ grepl("343-05", file_df_dedup$file_list),]

In [ ]:
file_df_dedup_tra <- file_df_dedup
save(file_df_dedup_tra, file = paste0(datapath,"capTCRseq/TRAfiles.RData"))

In [ ]:
dim(file_df)
dim(file_df_dedup)

In [ ]:
batchTRA <- immunelistfx(file_df_dedup$file_list, h4hpath, "TRA")

In [ ]:
names(batchTRA) <- gsub(".*CLONES_TRA", "", names(batchTRA))

In [ ]:
TRAlist <- batchTRA

In [ ]:
save(TRAlist, file = paste0(datapath,"capTCRseq/TRAlist.RData"))

### divstats

In [ ]:
load(file = paste0(datapath,"capTCRseq/TRAfiles.RData"))

In [ ]:
load(file = paste0(datapath,"capTCRseq/TRAlist.RData"))

In [ ]:
# remove lists with no clonotype
TRAlist <- TRAlist[sapply(TRAlist, function(x) length((x)) != 0)]

In [ ]:
length(TRAlist)

In [ ]:
Divstats.fx(TRAlist, "TRA", "CHP" , paste0(datapath, "capTCRseq/"))

### Cleanup divstats

In [ ]:
divstats <- read.csv(paste0(datapath, "capTCRseq/divstats_TRACHP.csv"), header = T, stringsAsFactors = F, row.names = 1)

In [ ]:
# add filenames to divstats
myfilenames <- file_df_dedup_tra[ file_df_dedup_tra$samplename %in% names(TRAlist),]

In [ ]:
dim(divstats)

In [ ]:
head(divstats)

In [ ]:
divstats$sample_id <- rownames(divstats)
divstats$basename <- rownames(divstats)

In [ ]:
# add filenames to divstats
divstats$file <- myfilenames$file_list[ match(divstats$sample_id, myfilenames$samplename)]

In [ ]:
dim(divstats)

Separate samples

In [ ]:
divstats$Sample <- NA
divstats$Sample[grepl("PBMC", rownames(divstats))] <- "PBMC"
divstats$Sample[grepl("cfDNA", rownames(divstats))] <- "cfDNA"
divstats$Sample[grepl("FFPE", rownames(divstats))] <- "Tumor"

Separate LFS and non-LFS

In [ ]:
divstats$Group <- NA
divstats$Group[grepl("LFS", rownames(divstats))] <- "LFS"
divstats$Group[grepl("CHP", rownames(divstats))] <- "CHP"

Get patient ids

In [ ]:
divstats$Patient <- NA
# CHP
divstats$Patient[divstats$Group == "CHP"] <- substr(rownames(divstats)[divstats$Group == "CHP"],1,7)
# Normal samples have either YO or MO
divstats$Patient[grepl("YO", rownames(divstats))] <- gsub("_PBMC_DNA", "", rownames(divstats)[grepl("YO", rownames(divstats))])
divstats$Patient[grepl("MO", rownames(divstats))] <- gsub("_PBMC_DNA", "", rownames(divstats)[grepl("MO", rownames(divstats))])
# LFS
divstats$Patient[divstats$Group == "LFS"] <- substr(rownames(divstats)[divstats$Group == "LFS"],1,12)

Get cycles. For LFS, Normals and Tumors recode to 00

In [ ]:
divstats$Cycle <- NA
divstats$Cycle[divstats$Group == "CHP"] <- substr(rownames(divstats)[divstats$Group == "CHP"],9,10)

divstats$Cycle[grepl("YO", rownames(divstats))] <- "00"
divstats$Cycle[grepl("MO", rownames(divstats))] <- "00"
divstats$Cycle[ divstats$Sample == "Tumor"] <- "00"
divstats$Cycle[ divstats$Group == "LFS"] <- "00"

In [ ]:
divstats$sample_id <- NA

# bind patient and cycle for PBMC and cfdna to generate sample id
divstats$sample_id[ divstats$Sample != "Tumor"] <- paste(divstats$Patient[ divstats$Sample != "Tumor"], "_", 
                                                          divstats$Cycle[ divstats$Sample != "Tumor"], sep = "")

# For tumor samples start with rownames because some of them are two biopsies from the same tumor
divstats$sample_id[ divstats$Sample == "Tumor"] <- gsub("-FFPE-DN.*", "", rownames(divstats)[ divstats$Sample == "Tumor"])

In [ ]:
divstats$sample_id <- gsub("-", "_", divstats$sample_id)

In [ ]:
divstats$Patient <- NULL
divstats$Cycle <- NULL
divstats$Group <- NULL
divstats$Sample <- NULL

In [ ]:
head(divstats)

In [ ]:
write.csv(divstats,
          paste0(datapath, "capTCRseq/divstats_TRACHP.csv"), row.names = TRUE)

# Test dataset

In [ ]:
mordpath <- "/Users/anabbi/Desktop/Sam/immpedcan/mixcr/Tcelllines/muldercelllines/mixcr/clones/"

In [ ]:
file_list <- list.files(mordpath, pattern = paste0("CLONES_", "TRB"), recursive = T)

In [ ]:
file_list

In [ ]:
file_list <- file_list[ file_list != 'CLONES_TRBUndetermined_S0.txt']

In [ ]:
file_list

In [ ]:
cells <- immunelistfx(file_list, mordpath, "TRB")

In [ ]:
cells

In [ ]:
myimmunelist_cells <- cells 

In [ ]:
save(myimmunelist_cells, file = paste0(datapath,"capTCRseq/myimmunelist_cells.RData"))

In [ ]:
Divstats.fx(myimmunelist_cells, "TRB", "Cells" , paste0(datapath, "capTCRseq/"))